# AES Plaintext Recovery (ResNet)
In this experiment, the residual network tries to guess the plaintext from the ciphertext, helped with ascii per-byte correction. 

## Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from dataset.datasets import AESDatasetCiphertextPlaintext
from pipeline import *

2023-05-22 10:38:23.400246: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Importing the dataset

In [20]:
data = AESDatasetCiphertextPlaintext(128, 'large')

train_labels, train_samples, test_labels, test_samples = data.get_data()

In [21]:
get_dataset_info(train_labels, train_samples, test_labels, test_samples)

===== Training Labels Shape: (2234530, 128)
===== Label Shape: (128,)
===== Training Samples Shape: (2234530, 128)
===== Sample Shape: (128,)
===== Testing Labels Shape: (957656, 128)
===== Testing Samples Shape: (957656, 128)


## Creating the model

In [22]:
# Imports
from keras import Sequential
from keras.layers import Input, Dense, BatchNormalization, LayerNormalization
from keras.optimizers import Adam

### Model hyperparameters
In this code block, we specify most parameters and hyperparameters that will be used in the training of the neural network.

Add customization here.

In [23]:
input_shape = np.shape(train_samples[0])

# output dimension
dim = len(train_labels[0])

# units per hidden layer
units = dim*16

loss_scc = 'sparse_categorical_crossentropy'
loss_mse = 'mse'
loss_bce = 'binary_crossentropy'
# 0.1 to 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.1,
    decay_steps=500,
    decay_rate=0.01)
optimizer = Adam(learning_rate=0.001)
metrics = ['accuracy', 'binary_accuracy']
epochs = 5
batch_size = 5000

### Model
In this code block, we create the model, according to the parameters and the topology we want to achieve. 
We then compile it specifying the optimizer, the loss and the metrics we want outputted.

Add customization here.

In [24]:
inputs = Input(shape=input_shape)
net = inputs

for _ in range(5):
    x1 = BatchNormalization()(inputs)
    x1 = Dense(units=units, activation='relu')(net)

    x2 = BatchNormalization()(x1)
    x2 = Dense(units=units, activation='relu')(inputs)

    net = Add()([x1, x2])
    
net = Dense(units=dim, activation='softmax')(net)

neural_network = Model(inputs, net)

# Summary
neural_network.summary()

# Compile model
neural_network.compile(optimizer=optimizer, loss=loss_mse, metrics=metrics)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 dense_22 (Dense)               (None, 2048)         264192      ['input_3[0][0]']                
                                                                                                  
 dense_23 (Dense)               (None, 2048)         264192      ['input_3[0][0]']                
                                                                                                  
 add_10 (Add)                   (None, 2048)         0           ['dense_22[0][0]',               
                                                                  'dense_23[0][0]']         

### Training
In this code block, we train the model. It outputs, for each epoch, the loss and metrics.

This block mostly stays the same.

In [25]:
history = train_model(neural_network, train_samples, train_labels, 
                      batch_size=batch_size, 
                      epochs=epochs)

Epoch 1/5
403/403 [==============================] - 854s 2s/step - loss: 0.4298 - accuracy: 0.0146 - binary_accuracy: 0.5559 - val_loss: 0.4386 - val_accuracy: 5.9073e-04 - val_binary_accuracy: 0.5468
Epoch 2/5
403/403 [==============================] - 825s 2s/step - loss: 0.4296 - accuracy: 0.0148 - binary_accuracy: 0.5559 - val_loss: 0.4386 - val_accuracy: 0.0229 - val_binary_accuracy: 0.5468
Epoch 3/5
403/403 [==============================] - 822s 2s/step - loss: 0.4296 - accuracy: 0.0138 - binary_accuracy: 0.5559 - val_loss: 0.4386 - val_accuracy: 5.4598e-04 - val_binary_accuracy: 0.5468
Epoch 4/5
403/403 [==============================] - 813s 2s/step - loss: 0.4296 - accuracy: 0.0145 - binary_accuracy: 0.5559 - val_loss: 0.4386 - val_accuracy: 0.0142 - val_binary_accuracy: 0.5468
Epoch 5/5
403/403 [==============================] - 806s 2s/step - loss: 0.4296 - accuracy: 0.0150 - binary_accuracy: 0.5559 - val_loss: 0.4385 - val_accuracy: 0.0028 - val_binary_accuracy: 0.5468


In [26]:
print("Loss: {}".format(history.history['loss']))
print("Validation Loss: {}".format(history.history['val_loss']))
print("Validation Accuracy: {}".format(history.history['val_accuracy']))

Loss: [0.42984527349472046, 0.4296095669269562, 0.4296090602874756, 0.42959967255592346, 0.429575115442276]
Validation Loss: [0.43858256936073303, 0.43858253955841064, 0.4385814666748047, 0.43855082988739014, 0.43854862451553345]
Validation Accuracy: [0.0005907282466068864, 0.02286834456026554, 0.0005459761014208198, 0.014195378869771957, 0.0028417608700692654]


### Testing
Here, we evaluate the neural network with the test data.

This block stays the same.

### Prediction
Here is where we use the network as an attack. We could skip the testing phase and use this as our own testing phase. Here we could add the text correction "layer" and calculate the actual score we want, maybe using binary accuracy probability as a metric.

Customization!!!

In [30]:
results = neural_network.evaluate(test_samples, test_labels, batch_size=batch_size)
print("Test loss: {}".format(results[0]))
print("Test accuracy: {}".format(results[1]))

192/192 [==============================] - 126s 655ms/step - loss: 0.4303 - accuracy: 0.0031 - binary_accuracy: 0.5552
Test loss: 0.43028995394706726
Test accuracy: 0.003134737256914377


In [31]:
pred_size = 1000
predictions = [predict_sample(neural_network, test_samples[i]) for i in range(pred_size)]

1/1 [==============================] - 0s 31ms/step


In [33]:
metrics = [correct_and_metrics((predictions[i], test_labels[i])) for i in range(pred_size)]

In [34]:
correct_bytes = 0
correct_predictions = 0
for m in metrics:
    correct_bytes += m[0]
    correct_predictions += m[1]
num_bytes = len(test_labels[0]) // 8
                             
print("Correct bytes: {}".format(correct_bytes))
print("Byte accuracy: {}".format(correct_bytes/(num_bytes*pred_size)))
print("Correct predictions: {}".format(correct_predictions))
print("Prediction accuracy: {}".format(correct_predictions/pred_size))

Correct bytes: 0
Byte accuracy: 0.0
Correct predictions: 0
Prediction accuracy: 0.0
